# PYTHON CON CPLEX II

<table><tr>
<td> <img src="img/python.png" alt="Drawing" style="width: 220px;"/> </td>
<td> <img src="img/add.png" alt="Drawing" style="width:150px;"/> </td>
<td> <img src="img/cplex.png" alt="Drawing" style="width: 250px;"/> </td>
<td> <img src="img/equal.png" alt="Drawing" style="width: 150px;"/> </td>
<td> <img src="img/laptop.png" alt="Drawing" style="width: 200px;"/> </td>
</tr></table>

En esta clase:
- Revisión TAREA CPLEX
- Cutstock Problem
- Como usar Callback - Ejemplo
- Bender's decomposition
- Como hacer heuristicas
- Paquetes para metaheuristicas

# Ejercicio: Orienteering problem (OP)

Recall the MIP formulation:

In the OP, a set of N vertices $i$ is given, each with a score $S_{i}$. The starting point (vertex 1) and the end point (vertex $N$) are fixed. The time $t_{ij}$ needed to travel from vertex $i$ to $j$ is known for all vertices. Not all vertices can be visited since the available time is limited to a given time budget $T_{max}$. The goal of the OP is to determine a path, limited by Tmax, that visits some of the vertices, in order to maxi- mise the total collected score. The scores are assumed to be entirely additive and each vertex can be visited at most once.
The OP can also be defined with the aid of a graph $G=(V,A)$ where $V=\{v_{1}, . . . , v_{N}\}$ is the vertex set and $A$ is the arc set. In this definition the nonnegative score $S_i$ is associated with each vertex $v_i$ 2 $V$ and the travel time $t_{ij}$ is associated with each arc $a_{ij}$ $A$. The OP consists of determining a Hamiltonian path $G^{’}$ $(G)$ over a sub-set of $V$, including preset start ($v_1$) and end ($v_N$) vertex, and having a length not exceeding $T_{max}$, in order to maximise the total collected score. In most cases, the OP is defined as a path to be found between distinct vertices, rather than a circuit or tour $(v_1,...,v_N)$. In many applications, however, $v_1$ does coincide with $v_N$. The difference between both formulations is small. It is always possible to add a dummy arc between end and start vertex to turn a path problem into a circuit problem. Mansini et al. (2006) explicitly define the ‘‘tour orienteering problem” as an OP where the start and end vertex coincide.
Making use of the notation introduced above, the OP can be for- mulated as an integer problem. The following decision variables are used: $x_{ij} = 1$ if a visit to vertex $i$ is followed by a visit to vertex $j$ – 0 otherwise; $u_i$ = the position of vertex i in the path.

<img src="http://www.sciweavers.org/tex2img.php?eq=Max%20%5Csum_%7Bi%3D2%7D%5E%7BN-1%7D%5Csum_%7Bj%3D2%7D%5E%7BN%7DS_i%20x_%7Bij%7D%20%5C%5C%20%5Csum_%7Bj%3D2%7D%5E%7BN%7Dx_%7B1j%7D%20%3D%20%5Csum_%7Bi%3D1%7D%5E%7BN-1%7D%20x_%7BiN%7D%20%3D%201%5C%5C%0A%5Csum_%7Bi%3D1%7D%5E%7BN-1%7Dx_%7Bik%7D%3D%5Csum_%7Bj%3D2%7D%5E%7BN%7Dx_%7Bkj%7D%20%5Cleq%201%2C%5Cforall%20k%3D2%2C...%2CN-1%5C%5C%0A%5Csum_%7Bi%3D1%7D%5E%7BN-1%7D%5Csum_%7Bj%3D2%7D%5E%7BN%7Dt_%7Bij%7Dx_%7Bij%7D%20%5Cleq%20T_%7Bmax%7D%5C%5C%202%20%5Cleq%20u_%7Bi%7D%20%5Cleq%20N%2C%5Cforall%20i%3D2%2C...%2CN%5C%5C%0Au_%7Bi%7D-u_%7Bj%7D%2B%201%20%5Cleq%20%28N-1%29%281-x_%7Bij%7D%29%2C%5Cforall%20i%2Cj%3D2%2C...%2CN%5C%5C%0Ax_%7Bij%7D%20%5Cin%20%7B0%2C1%7D%2C%5Cforall%20i%2Cj%3D1%2C...%2CN&bc=White&fc=Black&im=jpg&fs=12&ff=txfonts&edit=0" align="center" border="0" alt="Max \sum_{i=2}^{N-1}\sum_{j=2}^{N}S_i x_{ij} \\ \sum_{j=2}^{N}x_{1j} = \sum_{i=1}^{N-1} x_{iN} = 1\\\sum_{i=1}^{N-1}x_{ik}=\sum_{j=2}^{N}x_{kj} \leq 1,\forall k=2,...,N-1\\\sum_{i=1}^{N-1}\sum_{j=2}^{N}t_{ij}x_{ij} \leq T_{max}\\ 2 \leq u_{i} \leq N,\forall i=2,...,N\\u_{i}-u_{j}+ 1 \leq (N-1)(1-x_{ij}),\forall i,j=2,...,N\\x_{ij} \in \{0,1\},\forall i,j=1,...,N" width="300" height="296" />

The objective function is to maximise the total collected score. The first Constraints guarantee that the path starts in vertex 1 and ends in vertex N. The Second Constraints ensure the connectivity of the path and guarantee that every vertex is visited at most once. Constraint (3) ensures the limited time budget. Constraints (4) and (5) are nec- essary to prevent subtours. These subtour elimination constraints are formulated according to the Miller–Tucker–Zemlin (MTZ) for- mulation of the TSP (Miller et al., 1960).

[Pagina para instancias de OP](https://www.mech.kuleuven.be/en/cib/op#section-2)

# Lectura de Datos y Análisis

Leo los datos desde un txt

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
ruta = 'data/tsiligirides_problem_1_budget_85.txt'
archivo = pd.read_csv(ruta, header=None, delim_whitespace=True)

Asigno el tiempo o distancia máxima de la ruta

In [ ]:
tiempo_max=archivo.iloc[-1].values
tiempo_max=int(tiempo_max[0])

Creo matriz de distancias

In [ ]:
dist=[]
for i in range(31):
    a=archivo.iloc[i]
    a=np.array(a)
    a=np.delete(a,2)
    for j in range(31):
        if i!=j:
            b=archivo.iloc[j]
            b=np.array(b)
            b=np.delete(b,2)
            dist.append(int(np.linalg.norm(a-b)))
        elif i==j:
            dist.append(99999)

In [ ]:
distancias=np.array(dist)
distancias=distancias.reshape(31,31)
#print(distancias)

Obtengo los beneficios de cada nodo

In [ ]:
beneficios=[]
for i in range(31):
    c= archivo.iloc[i]
    c = [int(i) for i in c]
    del c[0]
    del c[0]
    c=c[0]
    beneficios.append(c)
print(beneficios)   

### EJERCICIO-> HACER LA IMPLEMENTACIÓN EN CPLEX CONECTADO A PYTHON

# DEAP biblioteca para hacer metaheuristicas en python

<img src="img/nsga2.jpg" alt="Drawing"/>

In [1]:
import random

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

IND_INIT_SIZE = 5
MAX_ITEM = 50
MAX_WEIGHT = 50
NBR_ITEMS = 20

In [2]:
# Para asegurar la reproductibilidad, la semilla RNG se coloca antes de la inicialización de los artículos. 
# También se siembra en main().
random.seed(64)

# Crear el diccionario de ítems: el nombre del ítem es un número entero, y el valor es 
# 2-tupla :  (peso, valor).
items = {}
# Crear elementos aleatorios y guárdelos en el diccionario de elementos.
for i in range(NBR_ITEMS):
    items[i] = (random.randint(1, 10), random.uniform(0, 100))

In [ ]:
creator.create("Fitness", base.Fitness, weights=(-1.0, 1.0))
creator.create("Individual", set, fitness=creator.Fitness)

In [3]:
toolbox = base.Toolbox()
# Generador de atributos
toolbox.register("attr_item", random.randrange, NBR_ITEMS)

# Inicializadores de estructura
toolbox.register("individual", tools.initRepeat, creator.Individual, 
                toolbox.attr_item, IND_INIT_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [4]:
def evalKnapsack(individual):
    weight = 0.0
    value = 0.0
    for item in individual:
        weight += items[item][0]
        value += items[item][1]
    if len(individual) > MAX_ITEM or weight > MAX_WEIGHT:
        return 10000, 0             # Se asegura que las mochilas con sobrepeso esten dominadas.
    return weight, value

In [5]:
def cxSet(ind1, ind2):
    """Aplica una operación crossover en los conjuntos de entradas. El primer 
    hijo es la intersección de los dos conjuntos, el segundo hijo es la 
    diferencia de los dos conjuntos.
    """
    temp = set(ind1)                # Usado para mantener el tipo
    ind1 &= ind2                    # Intersección (inplace)
    ind2 ^= temp                    # Diferencia simétrica (inplace)
    return ind1, ind2

<img src="img/itHWa.jpg" alt="Drawing"/>

In [6]:
def mutSet(individual):
    """Mutation that pops or add an element."""
    if random.random() < 0.5:
        if len(individual) > 0:     # No se puede eliminar un conjunto vacio
            individual.remove(random.choice(sorted(tuple(individual))))
    else:
        individual.add(random.randrange(NBR_ITEMS))
    return individual,

toolbox.register("evaluate", evalKnapsack)
toolbox.register("mate", cxSet)
toolbox.register("mutate", mutSet)
toolbox.register("select", tools.selNSGA2)

In [7]:
def main():
    random.seed(64)
    NGEN = 50
    MU = 50
    LAMBDA = 100
    CXPB = 0.7
    MUTPB = 0.2
    
    pop = toolbox.population(n=MU)
    hof = tools.ParetoFront()
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean, axis=0)
    stats.register("std", numpy.std, axis=0)
    stats.register("min", numpy.min, axis=0)
    stats.register("max", numpy.max, axis=0)
    
    algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, stats,
                              halloffame=hof)
    
    return pop, stats, hof

In [8]:
if __name__ == "__main__":
    main() 

gen	nevals	avg                        	std                      	min                      	max                        
0  	50    	[ 22.78       210.00877715]	[ 5.88316241 71.09309649]	[10.         49.69958685]	[ 38.         345.35491309]
1  	87    	[  9.96       145.20790666]	[ 10.61312395 139.1868852 ]	[0. 0.]                  	[ 45.         414.48478381]
2  	91    	[ 3.26       61.20087478]  	[  7.44797959 125.53892809]	[0. 0.]                  	[ 28.         414.48478381]
3  	88    	[ 4.6        84.51641114]  	[  8.57438044 140.51459866]	[0. 0.]                  	[ 28.         414.48478381]
4  	92    	[ 2.4        52.24310488]  	[  5.82065288 108.88598622]	[0. 0.]                  	[ 28.         414.48478381]
5  	87    	[ 3.66       74.97342258]  	[  6.99316809 127.02866009]	[0. 0.]                  	[ 28.         414.48478381]
6  	82    	[  5.3        111.43072783]	[  7.61117599 142.76899122]	[0. 0.]                  	[ 28.         414.48478381]
7  	90    	[ 3.38       76.37304048]

## Material en internet:

- [DEAP Package documentation](https://deap.readthedocs.io)
- [Curso de profesor Sergio Correa - Universidad de la Serena](https://www.youtube.com/channel/UCGyGH1nuNrR1C35kWN9BMAg/videos)
- [Documentación oficial (en inglés)](https://www.ibm.com/support/knowledgecenter/SSSA5P_12.7.1/ilog.odms.cplex.help/refpythoncplex/html/cplex.Cplex-class.html)